In [79]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split


In [80]:
data = pd.read_pickle("norm_train_data.uu")

In [81]:
data.shape

(807647, 36)

In [82]:
test_data = pd.read_pickle("norm_test_data.uu")

In [83]:
test_data.shape

(202166, 35)

In [84]:
def sentiment(x):
    if x == 'neu':
        return 0
    elif x == 'pos':
        return 1
    elif x == 'neg': 
        return -1

data['comment_sentiment'] = data['comment_sentiment'].apply(sentiment)
test_data['comment_sentiment'] = test_data['comment_sentiment'].apply(sentiment)

In [85]:
test_data.columns

Index(['index', 'label', 'comment', 'author', 'subreddit', 'score', 'ups',
       'downs', 'date', 'created_utc', 'parent_comment', 'lemmatized_comment',
       'lemmatized_parent_comment', 'clean_comment', 'clean_parent_comment',
       'lemmatized_clean_comment', 'lemmatized_clean_parent_comment',
       'vectorized_comment', 'vectorized_clean_comment',
       'vectorized_parent_comment', 'vectorized_clean_parent_comment',
       'cosine_similarity_dirty_comments', 'cosine_similarity_clean_comments',
       'word_count', 'punctuation_count', 'has_repeated', 'exclaim_count',
       'qns_mark_count', 'ellipses_mark_count', 'interjection_count',
       'laughter_words_count', 'capitalized_word_count',
       'partial_capital_word_count', 'emoticon_count', 'comment_sentiment'],
      dtype='object')

In [86]:
# test_data.rename(columns = {'dirty_cosine_similarity':'cosine_similarity_dirty_comments'}, inplace=True)

In [87]:
test_data = test_data[['label','clean_comment','cosine_similarity_dirty_comments','word_count','punctuation_count','has_repeated','exclaim_count','qns_mark_count','ellipses_mark_count','interjection_count','laughter_words_count','capitalized_word_count','partial_capital_word_count','emoticon_count','comment_sentiment']]

In [88]:
test_data.shape

(202166, 15)

In [89]:
data.columns

Index(['index', 'label', 'comment', 'author', 'subreddit', 'score', 'ups',
       'downs', 'date', 'created_utc', 'parent_comment', 'word_count',
       'punctuation_count', 'has_repeated', 'exclaim_count', 'qns_mark_count',
       'ellipses_mark_count', 'interjection_count', 'laughter_words_count',
       'capitalized_word_count', 'partial_capital_word_count',
       'emoticon_count', 'clean_comment', 'lemmatized_comment',
       'lemmatized_parent_comment', 'clean_parent_comment',
       'lemmatized_clean_comment', 'clean_lemmatized_parent_comment',
       'vectorized_comment', 'vectorized_clean_comment', 'vectorized_parent',
       'vectorized_clean_parent', 'cosine_similarity_dirty_comments',
       'cosine_similarity_clean_comments', 'comment_sentiment',
       'parent_comment_sentiment'],
      dtype='object')

In [90]:
data1 = data.drop(['author','subreddit','date','created_utc','vectorized_comment',
                  'vectorized_clean_comment','vectorized_parent','vectorized_clean_parent'], axis = 1)

In [91]:
data1.head()

,index,label,comment,score,ups,downs,parent_comment,word_count,punctuation_count,has_repeated,...,clean_comment,lemmatized_comment,lemmatized_parent_comment,clean_parent_comment,lemmatized_clean_comment,clean_lemmatized_parent_comment,cosine_similarity_dirty_comments,cosine_similarity_clean_comments,comment_sentiment,parent_comment_sentiment
0,417033,0,But it could mean a loss,1.0,1.0,0.0,We need to take some shots down the field. Bei...,0.002251,0.000000,0.0,...,But it could mean a loss,But could mean loss,We need take shot field . Being way fuck caref...,We need to take some shots down the field Bein...,But could mean loss,We need take shot field Being way fuck careful...,0.538655,0.552133,-1,pos
1,59081,0,Donald the reality TV star does care about TV ...,1.0,-1.0,-1.0,About 71 million watch final presidential deba...,0.004052,0.000102,0.0,...,Donald the reality TV star does care about TV ...,Donald reality TV star care TV rating .,About 71 million watch final presidential deba...,About 71 million watch final presidential deba...,Donald reality TV star care TV rating,About 71 million watch final presidential deba...,0.582335,0.579743,1,neg
2,5664,1,me irl,1.0,-1.0,-1.0,How much you want to bet that the artist was m...,0.000450,0.000000,0.0,...,me irl,irl,How much want bet artist make regular tiger fu...,How much you want to bet that the artist was m...,irl,How much want bet artist make regular tiger fu...,0.515965,0.510553,0,neg
3,366838,1,Can't have a discussion on one without the oth...,55.0,55.0,0.0,"And of course there are the ""what about circum...",0.004052,0.000306,0.0,...,Cant have a discussion on one without the othe...,"Ca n't discussion one without , right ?",And course `` circumcision ? '' asshole ca n't...,And of course there are the what about circumc...,Cant discussion one without right,And course circumcision assholes cant thing,0.533188,0.423121,0,neg
4,907940,1,That dude's friend's life is nowhere near as i...,1.0,1.0,0.0,Maybe he found out that one of his friends lik...,0.011256,0.000510,0.0,...,That dudes friends life is nowhere near as imp...,That dude 's friend 's life nowhere near impor...,Maybe find one friend like . He n't want embar...,Maybe he found out that one of his friends lik...,That dudes friend life nowhere near important ...,Maybe find one friend like He doesnt want emba...,0.496283,0.471443,-1,pos


In [92]:
#try SVM with just the cleaned comments + feature selections
data_try = data1[['label','clean_comment','cosine_similarity_dirty_comments','word_count','punctuation_count','has_repeated','exclaim_count','qns_mark_count','ellipses_mark_count','interjection_count','laughter_words_count','capitalized_word_count','partial_capital_word_count','emoticon_count','comment_sentiment']]
data_try.head(20)

,label,clean_comment,cosine_similarity_dirty_comments,word_count,punctuation_count,has_repeated,exclaim_count,qns_mark_count,ellipses_mark_count,interjection_count,laughter_words_count,capitalized_word_count,partial_capital_word_count,emoticon_count,comment_sentiment
0,0,But it could mean a loss,0.538655,0.002251,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,-1
1,0,Donald the reality TV star does care about TV ...,0.582335,0.004052,0.000102,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.001203,0.0,0.0,1
2,1,me irl,0.515965,0.000450,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,0
3,1,Cant have a discussion on one without the othe...,0.533188,0.004052,0.000306,0.0,0.000000,0.111111,0.0000,0.000000,0.0,0.000000,0.0,0.0,0
4,1,That dudes friends life is nowhere near as imp...,0.496283,0.011256,0.000510,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,-1
5,1,So antiMormon lies then,0.410225,0.001351,0.000612,0.0,0.000000,0.111111,0.0625,0.000000,0.0,0.000000,0.0,0.0,-1
6,1,Fiddlesticks rip off,0.472955,0.000900,0.000102,0.0,0.022727,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,0
7,1,I dont have neither an X1 nor an X360 so they ...,0.558923,0.006303,0.000306,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.001203,0.0,0.0,1
8,1,Yeah this would be so convenient to carry around,0.650354,0.003602,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,1
9,1,You forgot,0.505771,0.000450,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0,0


In [93]:
 #split
#  X_train, X_test, y_train, y_test = train_test_split(data_try.drop('label', axis = 1), data_try.label.astype(int), 
#                                                     stratify=data_try.label.astype(int), 
#                                                    random_state=42, 
#                                                    test_size=0.2, shuffle=True)

In [94]:
data_try.shape

(807647, 15)

# Hyperparameter Tuning via Cross Validation

In [95]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import LinearSVC

lsvc = LinearSVC()
tfidf_1 = TfidfVectorizer()
clmn = ColumnTransformer([("tfidf_1", tfidf_1, "clean_comment")
                         ],
                         remainder="passthrough")

pipe_cv = Pipeline([
                  ('tfidf', clmn),
                  ('classify', lsvc)
                ],verbose=True)

In [96]:
random_grid = {
    'classify__C': [0.1, 0.3, 0.5, 0.8, 1]
}

In [97]:
from sklearn.model_selection import RandomizedSearchCV

svc_random = RandomizedSearchCV(pipe_cv, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = 1)
search = svc_random.fit(data_try.drop('label', axis = 1),data_try.label)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   5.7s
[Pipeline] .......... (step 2 of 2) Processing classify, total=   3.5s
[CV] END ....................................classify__C=0.1; total time=  11.9s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   5.7s
[Pipeline] .......... (step 2 of 2) Processing classify, total=   3.6s
[CV] END ....................................classify__C=0.1; total time=  11.9s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   5.6s
[Pipeline] .......... (step 2 of 2) Processing classify, total=   3.6s
[CV] END ....................................classify__C=0.1; total time=  11.9s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   5.6s
[Pipeline] .......... (step 2 of 2) Processing classify, total=   6.8s
[CV] END ....................................classify__C=0.3; total time=  15.1s
[Pipeline] ............. (step 1 of 2) Processin

In [98]:
search.best_params_

{'classify__C': 0.1}

# Fitting svm (LinearSVC)

In [110]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
#TODO: tune

# model = SVC(C=1.0, probability=True)
# model = SVC(C=1.0)
lsvc = LinearSVC(C=0.1)
tfidf_1 = TfidfVectorizer()
clmn = ColumnTransformer([("tfidf_1", tfidf_1, "clean_comment")
                         ],
                         remainder="passthrough")

pipe = Pipeline([
                  ('tfidf', clmn),
                  ('classify', lsvc)
                ],verbose=True)



In [111]:
print(data_try.shape)
print(test_data.shape)

(807647, 15)
(202166, 15)


In [112]:
pipe.fit(data_try.drop('label', axis = 1),data_try.label) # 4 mins

[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   9.1s
[Pipeline] .......... (step 2 of 2) Processing classify, total=   5.9s


Pipeline(steps=[('tfidf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tfidf_1', TfidfVectorizer(),
                                                  'clean_comment')])),
                ('classify', LinearSVC(C=0.1))],
         verbose=True)

In [113]:
def metrics(test_data, prediction):
  cm = pd.DataFrame(confusion_matrix(test_data, prediction))
  cm.columns = ['Predicted Y=0','Predicted Y=1']
  cm.index = ['True Y=0','True Y=1']
  display(cm)

  accuracy = (cm.iloc[0,0]+cm.iloc[1,1])/(cm.iloc[0,0]+cm.iloc[1,1]+cm.iloc[0,1]+cm.iloc[1,0])
  print('Accuracy: '+str(accuracy))

  # Possible ways to further improve accuracy is to try adjusting the level of significance.

  # Calculate Sensitivity (true positive rate)
  sensitivity = cm.iloc[1, 1]/(cm.iloc[1, 1] + cm.iloc[1, 0])
  print('Sensitivity: '+ str(sensitivity))

  # Calculate Specificity (true negative rate)
  specificity = cm.iloc[0, 0]/(cm.iloc[0, 0] + cm.iloc[0, 1])
  print('Specificity: '+str(specificity))

  #Precision
  precision = cm.iloc[1,1]/(cm.iloc[1,1]+cm.iloc[0,1])
  print('Precision: '+str(precision))

  # AUC
  print('ROC-AUC:',roc_auc_score(test_data, prediction))

  #F1 Score
  print("F1 score:", round(f1_score(test_data, prediction), 4))

In [114]:
y_pred = pipe.predict(test_data.drop('label',axis=1))
metrics(test_data.label.astype(int),y_pred)

,Predicted Y=0,Predicted Y=1
True Y=0,73514,27441
True Y=1,33836,67375


Accuracy: 0.6968975990027997
Sensitivity: 0.665688512118248
Specificity: 0.7281858253677381
Precision: 0.7105868207897401
ROC-AUC: 0.696937168742993
F1 score: 0.6874


In [25]:
with open('y_pred_norm+features+tuning.npy', 'wb') as f:
    np.save(f, y_pred)

In [26]:
with open('y_pred_norm+features+tuning.npy', 'rb') as f:
    y_pred_norm_features_tuning = np.load(f)

In [27]:
y_pred_norm_features_tuning

array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

# Fitting Naive Bayes

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
#TODO: tune

# model = SVC(C=1.0, probability=True)
# mnb = MultinomialNB()
mnb = BernoulliNB()
tfidf_1 = CountVectorizer()
mm = MinMaxScaler()
clmn = ColumnTransformer([("tfidf_1", tfidf_1, "clean_comment")
                         ],
                         remainder="passthrough")

pipe_nb = Pipeline([
                  ('tfidf', clmn),
                  ('classify', mnb)
                ],verbose=True)


In [19]:
data_try.shape

(807647, 12)

In [26]:
pipe_nb.fit(data_try.drop('label', axis = 1),data_try.label) # fast

[Pipeline] ............. (step 1 of 2) Processing tfidf, total=   7.1s
[Pipeline] .......... (step 2 of 2) Processing classify, total=   0.2s


Pipeline(steps=[('tfidf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tfidf_1', CountVectorizer(),
                                                  'clean_comment')])),
                ('classify', BernoulliNB())],
         verbose=True)

In [27]:
y_pred = pipe_nb.predict(test_data.drop('label',axis=1))
metrics(test_data.label.astype(int),y_pred)

,Predicted Y=0,Predicted Y=1
True Y=0,71170,29785
True Y=1,33334,67877


Accuracy: 0.6877862746455883
Sensitivity: 0.6706484473031588
Specificity: 0.704967559803873
Precision: 0.6950195572484692
ROC-AUC: 0.6878080035535159
F1 score: 0.6826
